## Graph3: 🤖Bot with sqlite memory

In [1]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, START, END, add_messages
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
# from tavily import TavilyClient
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langchain_groq import ChatGroq
# from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

In [2]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# llm = ChatGroq(model="llama-3.1-8b-instant")

class State(TypedDict):
    messages: Annotated[list, add_messages]

# tav_key = os.environ["TAVILY_API_KEY"]


search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm_with_tools = llm.bind_tools(tools=tools)

sqlite_conn = sqlite3.connect("checkpoint.sqlite",  check_same_thread=False)
memory = SqliteSaver(sqlite_conn)

In [3]:
def chatbot(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def router(state:State):
    last_message = state["messages"][-1]

    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return "tool_call"
    else: 
        return "end"

In [4]:
tool_node = ToolNode(tools=tools)

graph = StateGraph(State)
graph.add_node("chatbot", chatbot)
graph.add_node("tools", tool_node)

graph.set_entry_point("chatbot")
graph.add_conditional_edges("chatbot", router, {"tool_call":"tools", "end":END})
graph.add_edge("tools", "chatbot")

app = graph.compile(checkpointer=memory)
config = {"configurable":{
    "thread_id":2
}}

In [6]:
while True:
    user_input = input("👤 You: ")
    if user_input.lower() == "exit":
        print("👋 Bye Bye!")
        break
    else:
        
        result = app.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)
        
        print(f"BOT🤖: {result['messages'][-1].content}")

BOT🤖: Yes, I remember your name is Bobby.
BOT🤖: Yes, I'm here. How can I help you?
BOT🤖: Okay, I'll try! What's up? Anything I can do for you today, *pal*? 😉
BOT🤖: Cheers to you too, mate! What can I do for you today? Lay it on me!
BOT🤖: Haha! Got it. Just letting me know you're still there? 👍 Anything else I can do for you? Maybe tell you a joke? Or look something up?
👋 Bye Bye!
